In [244]:
import pandas as pd
import numpy as np 
import json as js
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [259]:
barlettDB = pd.read_csv('all_pathogen.csv',sep=';')
print(barlettDB.shape)
barlettDB.loc[barlettDB['species'] == 'suis']

(1513, 10)


,superkingdom,phylum,class,order,family,genus,species,year,status,reference
349,Bacteria,Proteobacteria,Gammaproteobacteria,Pasteurellales,Pasteurellaceae,Actinobacillus,suis,1962,established,Taylor et al
392,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,suis,1966,established,Taylor et al
1209,Bacteria,Proteobacteria,Epsilonproteobacteria,Campylobacterales,Helicobacteraceae,Helicobacter,suis,2008,putative,"Joosten, Myrthe, et al. ""Case report: Helicoba..."


In [252]:
barlettDBE = barlettDB.loc[barlettDB['status'] == 'established']
print(barlettDBE.shape)
count_genus_b = barlettDB.groupby('genus').count().sort_values('species',ascending=False)


(1110, 10)


In [261]:
gen_sp_b = barlettDBE.loc[:,['genus','species']]
print(gen_sp_b.shape)
gen_sp_b = gen_sp_b.reset_index(drop=True)
gen_sp_b.loc[gen_sp_b['species'] == 'suis']

(1110, 2)


,genus,species
333,Actinobacillus,suis
374,Streptococcus,suis


In [262]:
def readGroupsNames():
    with open(os.path.normpath("pathogen-detection/data/groups/groups_name_id.json"),"r") as data:
        groups = js.load(data)
        return groups

In [263]:
pathogenDB = pd.DataFrame(columns=['genus','species','host'])

for group,id in readGroupsNames().items():        
        # checando se existe informação sobre
        if os.path.exists(f'pathogen-detection/data/groups_info/{group}/{group}_filtered.json'):
            # carregando informação
            with open(f'pathogen-detection/data/groups_info/{group}/{group}_filtered.json','r') as json:
                info = js.load(json)

            for entry in info:
                if len(entry) == 2:
                    continue
                pathogenDB.loc[len(pathogenDB.index)] = [entry['scientific_name'].split()[0].rstrip(']').lstrip('['),entry['scientific_name'].split()[1].rstrip(']').lstrip('['),entry['host']] 
print(pathogenDB.shape)

(13691, 3)


In [264]:
print('all entries PathogenDB: ', pathogenDB.shape)
pathogenHDB = pathogenDB.loc[pathogenDB['host'] == 'HUMAN']
print('only Human host: ', pathogenHDB.shape)
pathogenHDB = pathogenHDB.drop_duplicates(['genus','species'])
print('only Human rmDuplicates: ',pathogenHDB.shape)
pathogenDB = pathogenDB.drop_duplicates(['genus','species'])
print('PathogenDB no duplicates: ',pathogenDB.shape)
pathogenHDB = pathogenHDB.reset_index(drop=True)
pathogenDB = pathogenDB.reset_index(drop=True)

all entries PathogenDB:  (13691, 3)
only Human host:  (10519, 3)
only Human rmDuplicates:  (132, 3)
PathogenDB no duplicates:  (181, 3)


In [275]:
print(pathogenHDB.shape)
print(gen_sp_b.shape)

# Perform left outer join
merged = pd.merge(pathogenHDB, gen_sp_b, how='left', on=['genus', 'species'], indicator=True)

# Filter out the rows that are present in both dataframes
non_intersect_rows = merged[merged['_merge'] == 'left_only']

# Drop the '_merge' column
non_intersect_rows = non_intersect_rows.drop('_merge', axis=1)

# Output the non-intersect rows
pathogenHDB_only = non_intersect_rows
pathogenHDB_only

(132, 3)
(1110, 2)


,genus,species,host
2,Acinetobacter,pittii,HUMAN
10,Bacillus,sp.,HUMAN
11,Bacillus,paranthracis,HUMAN
12,Burkholderia,cenocepacia,HUMAN
28,Campylobacter,lanienae,HUMAN
30,Campylobacter,showae,HUMAN
34,Candida,auris,HUMAN
45,Peptacetobacter,hiranonis,HUMAN
47,Intestinibacter,bartlettii,HUMAN
51,Corynebacterium,singulare,HUMAN


In [305]:
barlettDB.loc[barlettDB['species'] == 'michiganensis'] # urinalis kobei infantis auris showae lanienae 

,superkingdom,phylum,class,order,family,genus,species,year,status,reference
